In [1]:
import featuretools as ft
import pandas as pd

es = ft.demo.load_mock_customer(return_entityset=True, random_seed=0)

es['transactions'].df.head()

,transaction_id,session_id,transaction_time,amount,product_id
298,298,1,2014-01-01 00:00:00,127.64,5
2,2,1,2014-01-01 00:01:05,109.48,2
308,308,1,2014-01-01 00:02:10,95.06,3
116,116,1,2014-01-01 00:03:15,78.92,4
371,371,1,2014-01-01 00:04:20,31.54,3


In [2]:
es['customers'].df

,customer_id,join_date,date_of_birth,zip_code
5,5,2010-07-17 05:27:50,1984-07-28,60091
4,4,2011-04-08 20:08:14,2006-08-15,60091
1,1,2011-04-17 10:48:33,1994-07-18,60091
3,3,2011-08-13 15:42:34,2003-11-21,13244
2,2,2012-04-15 23:31:04,1986-08-18,13244


Here, we have two time columns, join_date and date_of_birth. While either column might be useful for making features, the join_date should be used as the time index because it indicates when that customer first became available in the dataset.

The cutoff_time specifies the last point in time that a row’s data can be used for a feature calculation. Any data after this point in time will be filtered out before calculating features.

In [3]:
fm, features = ft.dfs(entityset=es,
                     target_entity='customers',
                     cutoff_time=pd.Timestamp('2014-1-1 04:00'),
                     instance_ids=[1,2,3], #customer id
                     cutoff_time_in_index=True)
fm

,,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,NUM_UNIQUE(sessions.WEEKDAY(session_start)),MODE(sessions.YEAR(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.MONTH(session_start)),MODE(sessions.DAY(session_start)),MODE(sessions.WEEKDAY(session_start)),NUM_UNIQUE(transactions.sessions.customer_id),NUM_UNIQUE(transactions.sessions.device),MODE(transactions.sessions.customer_id),MODE(transactions.sessions.device)
customer_id,time,,,,,,,,,,,,,,,,,,,,,
1,2014-01-01 04:00:00,60091,4,3,tablet,4958.19,42.309717,139.23,-0.006928,5.81,74.002836,...,1,2014,4,1,1,2,1,3,1,tablet
2,2014-01-01 04:00:00,13244,4,2,desktop,4150.30,39.289512,146.81,-0.134786,12.07,84.700000,...,1,2014,2,1,1,2,1,2,2,desktop
3,2014-01-01 04:00:00,13244,1,1,tablet,941.87,47.264797,146.31,0.618455,8.19,62.791333,...,1,2014,1,1,1,2,1,1,3,tablet


In [4]:
cutoff_times = pd.DataFrame()
cutoff_times['customer_id'] = [1,2,3,1]
cutoff_times['time']=pd.to_datetime(['2014-1-1 04:00',
                                    '2014-1-1 05:00',
                                    '2014-1-1 06:00',
                                    '2014-1-1 08:00'])
cutoff_times['label'] = [True, True, False, True]
cutoff_times

,customer_id,time,label
0,1,2014-01-01 04:00:00,True
1,2,2014-01-01 05:00:00,True
2,3,2014-01-01 06:00:00,False
3,1,2014-01-01 08:00:00,True


In [5]:
fm, features = ft.dfs(entityset=es,
                     target_entity='customers',
                     cutoff_time=cutoff_times,
                     cutoff_time_in_index=True)
fm

,,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,MODE(sessions.YEAR(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.MONTH(session_start)),MODE(sessions.DAY(session_start)),MODE(sessions.WEEKDAY(session_start)),NUM_UNIQUE(transactions.sessions.customer_id),NUM_UNIQUE(transactions.sessions.device),MODE(transactions.sessions.customer_id),MODE(transactions.sessions.device),label
customer_id,time,,,,,,,,,,,,,,,,,,,,,
1,2014-01-01 04:00:00,60091,4,3,tablet,4958.19,42.309717,139.23,-0.006928,5.81,74.002836,...,2014,4,1,1,2,1,3,1,tablet,True
2,2014-01-01 05:00:00,13244,5,2,desktop,5155.26,38.047944,146.81,-0.121811,12.07,83.149355,...,2014,2,1,1,2,1,2,2,desktop,True
3,2014-01-01 06:00:00,13244,4,2,desktop,2867.69,40.349758,146.31,0.318315,6.65,65.174773,...,2014,1,1,1,2,1,2,3,desktop,False
1,2014-01-01 08:00:00,60091,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,...,2014,4,1,1,2,1,3,1,mobile,True


By default, all data up to and including the cutoff time is used. We can restrict the amount of historical data that is selected for calculations using a “training window.”

In [6]:
window_fm, window_features = ft.dfs(entityset=es,
                                   target_entity='customers',
                                   cutoff_time=cutoff_times,
                                   cutoff_time_in_index=True,
                                    training_window='2 hour')
window_fm

,,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,MODE(sessions.YEAR(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.MONTH(session_start)),MODE(sessions.DAY(session_start)),MODE(sessions.WEEKDAY(session_start)),NUM_UNIQUE(transactions.sessions.customer_id),NUM_UNIQUE(transactions.sessions.device),MODE(transactions.sessions.customer_id),MODE(transactions.sessions.device),label
customer_id,time,,,,,,,,,,,,,,,,,,,,,
1,2014-01-01 04:00:00,60091,2,2,desktop,2077.66,43.772157,139.09,-0.187686,5.81,76.950370,...,2014,1,1,1,2,1,2,1,desktop,True
2,2014-01-01 05:00:00,13244,3,2,desktop,2605.61,36.077146,146.81,-0.198611,12.07,84.051935,...,2014,1,1,1,2,1,2,2,desktop,True
3,2014-01-01 06:00:00,13244,3,1,desktop,1925.82,37.130891,128.26,0.110145,6.65,66.407586,...,2014,1,1,1,2,1,1,3,desktop,False
1,2014-01-01 08:00:00,60091,3,2,mobile,3124.15,38.952172,139.43,0.047120,5.91,66.471277,...,2014,1,1,1,2,1,2,1,mobile,True


In [7]:
fm[['COUNT(transactions)']]

,,COUNT(transactions)
customer_id,time,
1,2014-01-01 04:00:00,67
2,2014-01-01 05:00:00,62
3,2014-01-01 06:00:00,44
1,2014-01-01 08:00:00,126


In [8]:
window_fm[["COUNT(transactions)"]]

,,COUNT(transactions)
customer_id,time,
1,2014-01-01 04:00:00,27
2,2014-01-01 05:00:00,31
3,2014-01-01 06:00:00,29
1,2014-01-01 08:00:00,47


# Setting a Last Time Index

The training window in Featuretools limits the amount of past data that can be used while calculating a particular feature vector. A row in the entity is filtered out if the value of its time index is either before or after the training window. This works for entities where a row occurs at a single point in time. However, a row can sometimes exist for a duration.

For example, a customer’s session has multiple transactions which can happen at different points in time. If we are trying to count the number of sessions a user has in a given time period, we often want to count all the sessions that had any transaction during the training window. To accomplish this, we need to not only know when a session starts, but also when it ends. The last time that an instance appears in the data is stored as the last_time_index of an Entity. We can compare the time index and the last time index of the sessions entity above:

In [9]:
es['sessions'].df['session_start'].head()

1   2014-01-01 00:00:00
2   2014-01-01 00:17:20
3   2014-01-01 00:28:10
4   2014-01-01 00:44:25
5   2014-01-01 01:11:30
Name: session_start, dtype: datetime64[ns]

In [10]:
es['sessions'].last_time_index.head()

1   2014-01-01 00:16:15
2   2014-01-01 00:27:05
3   2014-01-01 00:43:20
4   2014-01-01 01:10:25
5   2014-01-01 01:22:20
Name: last_time, dtype: datetime64[ns]

Featuretools can automatically add last time indexes to every Entity in an Entityset by running EntitySet.add_last_time_indexes(). If a last_time_index has been set, Featuretools will check to see if the last_time_index is after the start of the training window. That, combined with the cutoff time, allows DFS to discover which data is relevant for a given training window.

# Approximating Features by Rounding Cutoff Times

For each unique cutoff time, Featuretools must perform operations to select the data that’s valid for computations. If there are a large number of unique cutoff times relative to the number of instances for which we are calculating features, the time spent filtering data can add up. By reducing the number of unique cutoff times, we minimize the overhead from searching for and extracting data for feature calculations.

One way to decrease the number of unique cutoff times is to round cutoff times to an earlier point in time. An earlier cutoff time is always valid for predictive modeling — it just means we’re not using some of the data we could potentially use while calculating that feature. So, we gain computational speed by losing a small amount of information.

To understand when an approximation is useful, consider calculating features for a model to predict fraudulent credit card transactions. In this case, an important feature might be, “the average transaction amount for this card in the past”. While this value can change every time there is a new transaction, updating it less frequently might not impact accuracy.

In [11]:
#fm = ft.calculate_feature_matrix(features=features,
#                                entityset=es,
#                                cutoff_time=ct,
#                                approximate='1 day')

# Secondary Time index

It is sometimes the case that information in a dataset is updated or added after a row has been created. This means that certain columns may actually become known after the time index for a row. Rather than drop those columns to avoid leaking information, we can create a secondary time index to indicate when those columns become known.

The Flights entityset is a good example of a dataset where column values in a row become known at different times. Each trip is recorded in the trip_logs entity, and has many times associated with it.

In [12]:
es_flight = ft.demo.load_flight(nrows=100)

In [13]:
es_flight

Entityset: Flight Data
  Entities:
    trip_logs [Rows: 100, Columns: 21]
    flights [Rows: 13, Columns: 9]
    airlines [Rows: 1, Columns: 1]
    airports [Rows: 6, Columns: 3]
  Relationships:
    trip_logs.flight_id -> flights.flight_id
    flights.carrier -> airlines.carrier
    flights.dest -> airports.dest

In [14]:
es_flight['trip_logs'].df.head(3)

,trip_log_id,flight_id,date_scheduled,scheduled_dep_time,scheduled_arr_time,dep_time,arr_time,dep_delay,taxi_out,taxi_in,arr_delay,scheduled_elapsed_time,air_time,distance,carrier_delay,weather_delay,national_airspace_delay,security_delay,late_aircraft_delay,canceled,diverted
30,30,AA-494:RSW->CLT,2016-09-03,2017-01-01 13:14:00,2017-01-01 15:05:00,2017-01-01 13:03:00,2017-01-01 14:53:00,-11.0,12.0,10.0,-12.0,6660000000000,88.0,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,38,AA-495:ATL->PHX,2016-09-03,2017-01-01 11:30:00,2017-01-01 15:40:00,2017-01-01 11:24:00,2017-01-01 15:41:00,-6.0,28.0,5.0,1.0,15000000000000,224.0,1587.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,46,AA-495:CLT->ATL,2016-09-03,2017-01-01 09:25:00,2017-01-01 10:42:00,2017-01-01 09:23:00,2017-01-01 10:39:00,-2.0,18.0,8.0,-3.0,4620000000000,50.0,226.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


For every trip log, the time index is date_scheduled, which is when the airline decided on the scheduled departure and arrival times, as well as what route will be flown. We don’t know the rest of the information about the actual departure/arrival times and the details of any delay at this time. However, it is possible to know everything about how a trip went after it has arrived, so we can use that information at any time after the flight lands.

Using a secondary time index, we can indicate to Featuretools which columns in our flight logs are known at the time the flight is scheduled, plus which are known at the time the flight lands.



In [15]:
es = ft.EntitySet('Flight Data')
arr_time_columns = ['arr_delay', 'dep_delay', 'carrier_delay', 'weather_delay',
                    'national_airspace_delay', 'security_delay',
                    'late_aircraft_delay', 'canceled', 'diverted',
                    'taxi_in', 'taxi_out', 'air_time', 'dep_time']
es.entity_from_dataframe('trip_logs',
                        data,
                        index='trip_log_id',
                        make_index=True,
                        time_index='date_schedule',
                        secondary_time_index={'arr_time':arr_time_columns})

NameError: name 'data' is not defined

Let’s make some features at varying times using the flight example described above. Trip 14 is a flight from CLT to PHX on January 31, 2017 and trip 92 is a flight from PIT to DFW on January 1. We can set any cutoff time before the flight is scheduled to depart, emulating how we would make the prediction at that point in time.

We set two cutoff times for trip 14 at two different times: one which is more than a month before the flight and another which is only 5 days before. For trip 92, we’ll only set one cutoff time, three days before it is scheduled to leave.

In [17]:
ct_flight = pd.DataFrame()
ct_flight['trip_log_id'] = [14, 14, 92]
ct_flight['time'] = pd.to_datetime(['2016-12-28',
                                    '2017-1-25',
                                    '2016-12-28'])
ct_flight['label'] = [True, True, False]
ct_flight

,trip_log_id,time,label
0,14,2016-12-28,True
1,14,2017-01-25,True
2,92,2016-12-28,False


In [18]:
fm, features = ft.dfs(entityset=es_flight,
                     target_entity='trip_logs',
                     cutoff_time=ct_flight,
                     cutoff_time_in_index=True,
                     agg_primitives=['max'],
                     trans_primitives=['month'])

In [19]:
fm[['flight_id', 'label', 'flights.MAX(trip_logs.arr_delay)', 'MONTH(scheduled_dep_time)']]

flight_id  label  \
trip_log_id time                                 
14          2016-12-28  AA-494:CLT->PHX   True   
            2017-01-25  AA-494:CLT->PHX   True   
92          2016-12-28  AA-496:PIT->DFW  False   

                        flights.MAX(trip_logs.arr_delay)  \
trip_log_id time                                           
14          2016-12-28                               NaN   
            2017-01-25                              33.0   
92          2016-12-28                               NaN   

                        MONTH(scheduled_dep_time)  
trip_log_id time                                   
14          2016-12-28                          1  
            2017-01-25                          1  
92          2016-12-28                          1

# Creating and Flattening a Feature Tensor
The make_temporal_cutoffs() function generates a series of equally spaced cutoff times from a given set of cutoff times and instance ids.

This function can be paired with DFS to create and flatten a feature tensor rather than making multiple feature matrices at different delays.

The function takes in the the following parameters:

instance_ids (list, pd.Series, or np.ndarray): A list of instances.

cutoffs (list, pd.Series, or np.ndarray): An associated list of cutoff times.

window_size (str or pandas.DateOffset): The amount of time between each cutoff time in the created time series.

start (datetime.datetime or pd.Timestamp): The first cutoff time in the created time series.

num_windows (int): The number of cutoff times to create in the created time series.

Only two of the three options window_size, start, and num_windows need to be specified to uniquely determine an equally-spaced set of cutoff times at which to compute each instance.

In [20]:
cutoff_times


,customer_id,time,label
0,1,2014-01-01 04:00:00,True
1,2,2014-01-01 05:00:00,True
2,3,2014-01-01 06:00:00,False
3,1,2014-01-01 08:00:00,True


Then passing in window_size='1h' and num_windows=2 makes one row an hour over the last two hours to produce the following new dataframe. The result can be directly passed into DFS to make features at the different time points.

In [21]:
temporal_cutoffs = ft.make_temporal_cutoffs(cutoff_times['customer_id'],
                                           cutoff_times['time'],
                                           window_size='1h',
                                           num_windows=2)

temporal_cutoffs

,time,instance_id
0,2014-01-01 03:00:00,1
1,2014-01-01 04:00:00,1
2,2014-01-01 04:00:00,2
3,2014-01-01 05:00:00,2
4,2014-01-01 05:00:00,3
5,2014-01-01 06:00:00,3
6,2014-01-01 07:00:00,1
7,2014-01-01 08:00:00,1


In [23]:
es = ft.demo.load_mock_customer(return_entityset=True, random_seed=0)

fm, features = ft.dfs(entityset=es,
            target_entity='customers',
            cutoff_time=temporal_cutoffs,
            cutoff_time_in_index=True)

In [24]:
fm

zip_code  COUNT(sessions)  \
customer_id time                                            
1           2014-01-01 03:00:00    60091                3   
            2014-01-01 04:00:00    60091                4   
2           2014-01-01 04:00:00    13244                4   
            2014-01-01 05:00:00    13244                5   
3           2014-01-01 05:00:00    13244                2   
            2014-01-01 06:00:00    13244                4   
1           2014-01-01 07:00:00    60091                7   
            2014-01-01 08:00:00    60091                8   

                                 NUM_UNIQUE(sessions.device)  \
customer_id time                                               
1           2014-01-01 03:00:00                            3   
            2014-01-01 04:00:00                            3   
2           2014-01-01 04:00:00                            2   
            2014-01-01 05:00:00                            2   
3           2014-01-01 05:00:00                            2   
            2014-01-01 06:00:00                            2   
1           2014-01-01 07:00:00                            3   
            2014-01-01 08:00:00                            3   

                                MODE(sessions.device)  \
customer_id time                                        
1           2014-01-01 03:00:00               desktop   
            2014-01-01 04:00:00                tablet   
2           2014-01-01 04:00:00               desktop   
            2014-01-01 05:00:00               desktop   
3           2014-01-01 05:00:00               desktop   
            2014-01-01 06:00:00               desktop   
1           2014-01-01 07:00:00                tablet   
            2014-01-01 08:00:00                mobile   

                                 SUM(transactions.amount)  \
customer_id time                                            
1           2014-01-01 03:00:00                   3932.56   
            2014-01-01 04:00:00                   4958.19   
2           2014-01-01 04:00:00                   4150.30   
            2014-01-01 05:00:00                   5155.26   
3           2014-01-01 05:00:00                   1886.72   
            2014-01-01 06:00:00                   2867.69   
1           2014-01-01 07:00:00                   7605.53   
            2014-01-01 08:00:00                   9025.62   

                                 STD(transactions.amount)  \
customer_id time                                            
1           2014-01-01 03:00:00                 42.769602   
            2014-01-01 04:00:00                 42.309717   
2           2014-01-01 04:00:00                 39.289512   
            2014-01-01 05:00:00                 38.047944   
3           2014-01-01 05:00:00                 41.199361   
            2014-01-01 06:00:00                 40.349758   
1           2014-01-01 07:00:00                 41.018896   
            2014-01-01 08:00:00                 40.442059   

                                 MAX(transactions.amount)  \
customer_id time                                            
1           2014-01-01 03:00:00                    139.23   
            2014-01-01 04:00:00                    139.23   
2           2014-01-01 04:00:00                    146.81   
            2014-01-01 05:00:00                    146.81   
3           2014-01-01 05:00:00                    146.31   
            2014-01-01 06:00:00                    146.31   
1           2014-01-01 07:00:00                    139.43   
            2014-01-01 08:00:00                    139.43   

                                 SKEW(transactions.amount)  \
customer_id time                                             
1           2014-01-01 03:00:00                   0.140387   
            2014-01-01 04:00:00                  -0.006928   
2           2014-01-01 04:00:00                  -0.134786   
            2014-01-01 05:00:00                  -0.121811   
3   